# LLM's Experiments in Humor

This notebook contains my observations on various jokes and a description of the techniques I used. You can find all relevant links in the README.

In [56]:
from openai import OpenAI, AsyncOpenAI
from enum import Enum
import json
import csv
import re

OPENAI_API_KEY = "YOUR_API_KEY"

In [6]:
class OpenAIModel(Enum):
    GPT35 = "gpt-3.5-turbo-0125"  # 0.50 USD per 1 million tokens
    GPT4 = "gpt-4-turbo"  # 10 USD per 1 million tokens
    GPT4o = "gpt-4o"  # 5 USD per 1 million tokens


In [24]:
class OpenAILLM():

    def __init__(self, model: OpenAIModel):
        self.model = model
        self.openai = OpenAI(api_key=OPENAI_API_KEY)
        self.input_tokens = 0
        self.output_tokens = 0

    def answer(self, query: str, temperature: float) -> str:
        response = self.openai.chat.completions.create(
            model=self.model.value,
            messages=[
                {
                    "role": "user",
                    "content": query
                }
            ],
            temperature=temperature
        )
        self.input_tokens += response.usage.prompt_tokens
        self.output_tokens += response.usage.completion_tokens
        return response.choices[0].message.content



Different strategies

In [47]:
class PromptFormat(Enum):
    BASE = "base_prompt"
    AUDIENCE = "audience_prompt"
    RULE_OF_THREE = "rule_of_three_prompt"
    PUN = "pun_prompt"
    MISPLACED_SINCERITY = "misplaced_sincerity_prompt"
    BROKEN_ASSUMPTIONS = "broken_assumptions_prompt"

I used the default temperature setting here, although experimenting with different temperatures might be a good way to enhance the creativity of the LLM.

In [36]:
model = OpenAIModel.GPT4o
llm = OpenAILLM(model)

jokes = {}

def generate_jokes(prompt, format):
    jokes[format] = []
    
    for i in range(3):
        joke = llm.answer(prompt, temperature=0.7)
        jokes[format].append(joke)

In [45]:
def print_jokes(format):
    print(f"{format}: ")
    for joke in jokes[format]:
        print(joke)
    

# Base prompt

I will start with the most basic prompt, which literally asks LLM to generate a joke about linux without any additional information. After that, I will make a more complex prompt, and after all evaluate the generated prompts. For each technique, we will generate 3 jokes and put them into the map container.

In [73]:
base_prompt = "Generate a original joke about Linux."
generate_jokes(base_prompt, PromptFormat.BASE)

print(jokes[PromptFormat.BASE])


["Why did the Linux server go broke?\n\nBecause it couldn't find a CentOS to save!", 'Why did the Linux kernel go to therapy?\n\nBecause it had too many unresolved dependencies!', "Why did the Linux user break up with their Windows computer?\n\nBecause they couldn't handle the constant neediness and preferred something open to new possibilities!"]


# Audience 

The same joke can attract different reactions from different groups of people—while one group may respond with silent stares, another might laugh out loud. So one important question is:  Does the audience relate? Before employing different techniques, I decided to use a template that describes the audience for whom the joke is intended.


In [74]:
audience_prompt = "Generate an original joke about Linux to share with an audience of tech enthusiasts and professionals."
generate_jokes(audience_prompt, PromptFormat.AUDIENCE)

print_jokes(PromptFormat.AUDIENCE)

PromptFormat.AUDIENCE: 
Why do Linux users never need a therapist?

Because they’re always great at handling their own partitions!
Why do Linux users always bring a ladder to their computer?

Because they heard their system is running on "elevated" privileges!
Why do Linux users never get lost?

Because they always know how to find their root directory!


# Different Strategies

## Rule of Three

**Definition** The Rule of Three joke format is when you have a list of two similar-
objects/actions and one that doesn’t belong. The first two objects/actions
create a pattern. The third breaks that pattern.

* SAFETY: The 1st and 2nd items in a list (these items share something in
common)

* VIOLATION: The 3rd item in a list (which is very different from the first 2)

* COMEDIC CONFLICT: Comedic conflict comes from breaking the
audience’s assumptions about what the third item in the list will be

In [37]:
rule_of_three_prompt = """
    Create an original Linux joke for an audience of tech enthusiasts and professionals using the Rule of Three structure. This format involves listing three elements: the first two are similar and establish a pattern, while the third element unexpectedly breaks this pattern, adding a humorous twist.

    Example of the technique:
    3 men were on a deserted island and found a genie. The genie
gave each one wish. The first wished he could be back home
with his family and poof! He was gone. The second said he
wished he could be back in his hometown and poof! He was
gone. The third person’s wish was “I want my friends back.'
"""
format = PromptFormat.RULE_OF_THREE
generate_jokes(rule_of_three_prompt, format)

print_jokes(format)

${format}: 
Three sysadmins walk into a server room. The first one says, "I just patched the kernel and everything's running smoothly." The second one says, "I just optimized the database, and the queries are lightning-fast." The third one says, "I just ran 'rm -rf /' and—hey, where did everyone go?"
Why did the Linux admin bring a laptop, a server, and a cat to the data center?

The laptop was for remote management, the server was for deploying new services, and the cat was to "tail" the log files.
Three Linux sysadmins walked into a bar. The first ordered a drink and asked the bartender to chmod 755. The second ordered a drink and asked the bartender to chown root:root. The third sysadmin ordered a drink and asked the bartender to alias 'rm' to 'echo You wish!'.


# Pun / Double-Entendre

The double-entendre, better known as a pun, is often said to be the lowest
form of humor. The reason for this is because coming up with a corny pun
is extremely easy. You can take almost any word in the English language,
find an alternate meaning, and then create a pun from it. (It should be super easy for LLM)


In [41]:
pun_prompt = """Create an original Linux joke for an audience of tech enthusiasts and professionals using a pun or double entendre. Choose a word or phrase that has multiple meanings or sounds similar to another word. Create a setup that leads the audience to think about one meaning of the word, and then deliver a punchline that reveals the other meaning, providing a humorous twist. The joke should be clear and concise, ensuring the pun or double entendre is easily understandable and amusing.

Example #1: "I'm reading a book on anti-gravity. It's impossible to put down!"
Exmaple #2: "You can tune a guitar, but you can't tuna fish. Unless of course, you play bass."
"""

format = PromptFormat.PUN
generate_jokes(pun_prompt, format)

print_jokes(format)

PromptFormat.PUN: 
Why do Linux sysadmins never get lost? 

Because they always know their "root"!
Why do Linux servers never get tired?

Because they always have plenty of reboots!
Why did the Linux admin bring a ladder to the data center?

Because they heard the servers were experiencing high "uptime"!


# Misplaced Sincerity

Misplaced Sincerity  is where the comedian pretends to lack a certain knowledge or awareness of what the audience knows to be true. 

In [44]:
misplaced_sincerity_prompt = """Create an original Linux joke for an audience of tech enthusiasts and professionals using the technique of misplaced sincerity, where you, as the comedian, pretend not to understand something that the audience knows well. Start with a common phrase or situation. Then, deliberately misinterpret it in a humorous way that showcases your 'lack' of understanding, while the audience is in on the true meaning. The joke should highlight this comedic conflict by allowing the audience to see both the incorrect interpretation and the commonly understood meaning simultaneously.

Example: "Imagine you are talking about cooking and say, 'I was told to whip the cream, so I gave it a motivational speech to get it going. Apparently, that’s not what they meant?"
"""

format = PromptFormat.MISPLACED_SINCERITY
generate_jokes(misplaced_sincerity_prompt, format)

print_jokes(format)

PromptFormat.MISPLACED_SINCERITY: 
Alright, so I was trying to get into this whole Linux thing the other day, and someone told me, "You gotta get comfortable using the terminal."

So, you know, I went all out. I set up a little workbench, placed a terminal up there, and even got a cushion for it. I mean, if I'm going to be using this terminal a lot, it better be comfy, right?

Then they told me I needed to learn about "bash scripting." So there I was, looking up how to make a bash sound track. Turns out, it's not about sound effects at all! Who knew?

And don't even get me started on "man pages"—I thought they were some kind of Linux-themed comic book for men. Imagine my disappointment when I saw it was just documentation!
Sure, here's a Linux-themed joke using misplaced sincerity:

"Hey folks, so I was trying to get into the whole Linux thing, and someone told me I needed to use the 'root' account for administrative tasks. So there I was, digging in my garden, trying to find this 'roo

# Broken assumptions

This technique involves providing explicit and implicit information in a setup, where the audience fills in gaps with their assumptions. The punchline then subverts these assumptions, creating humor by shifting the expected understanding of the situation.

In [48]:
broken_assumptions_prompt = """Create an original Linux joke for an audience of tech enthusiasts and professionals that plays with their expectations by using the 'Broken Assumptions' technique. Begin by setting up a scenario that provides explicit information, leading the audience to form specific implicit assumptions about the 'who', 'what', 'where', 'when', 'why', and 'how' of the story. Then, deliver a punchline that unexpectedly violates one of these assumptions, creating a humorous twist.

Example: "I gave my cat a bath the other day… they love it. He sat there,
he enjoyed it, and it was fun for me too. The fur would stick to
my tongue, but other than that..."
"""

format = PromptFormat.BROKEN_ASSUMPTIONS
generate_jokes(broken_assumptions_prompt, format)

print_jokes(format)

PromptFormat.BROKEN_ASSUMPTIONS: 
Sure, here’s an original Linux joke for tech enthusiasts:

---

So, last weekend, my friend tells me he’s got this old laptop that’s just gathering dust and he wants to revive it. I suggest, “Why not install a lightweight Linux distro? It’ll breathe new life into that ancient machine!” He agrees, so I head over to help him out.

We spend the whole afternoon setting it up. We partition the drive, install Xubuntu, configure the settings, and finally, it’s running smoothly. He’s super excited and says, “Wow, this is amazing! It feels like a brand-new laptop!”

Just as I’m feeling pretty proud of our work, he scratches his head and asks, “So, when do we install Windows 98?”

---

The humor lies in the unexpected twist that despite all the setup and excitement about Linux, the friend’s concept of “reviving” the laptop was to install an ancient version of Windows, completely breaking the expectation that he understood and was on board with the idea of using 

# Rate jokes

I use a prompt to evaluate and rate the jokes based on how funny they are.

In [69]:
def rate_joke(joke):
    return f"Rate the funniness of the following joke on a scale from 1 to 5, where 1 is not funny at all and 5 is extremely funny: '{joke}'. Please provide only a single number as your response."


In [75]:
def json_to_csv(joke, rating, filename):
    
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        
        file.seek(0, 2)
        if file.tell() == 0:
            writer.writerow(['Joke Text', 'Rating']) 

       
        writer.writerow([joke, rating])

csv_file = 'joke_ratings.csv'


In [76]:
for format in jokes:
    for joke in jokes[format]:
        rating = llm.answer(rate_joke(joke),temperature=0.7)
        json_to_csv(joke, rating, csv_file)